In [8]:
import networkx as nx
from pylab import *
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [9]:
#some settings for the plot
matplotlib.rcParams.update({'font.size': 18, 'font.family': 'Arial', 'mathtext.fontset': 'stix'})
plt.rcParams['figure.dpi'] = 300
sns.set(context='paper', style='ticks')

TickSize = 11
LabelSize = 13
Labelpad = 5
Linewidth = 4

In [10]:
def getRobustness(g, sol, method):
    N = nx.number_of_nodes(g)
    Robust = [1]
    norm = N*(N-1)/2
    g_copy = g.copy()
    for i in range(len(sol)-1):
        g_copy.remove_node(sol[i])
        if method == 'ND':
            Gc = max(nx.connected_component_subgraphs(g_copy), key=len)
            robust = nx.number_of_nodes(Gc) / N
        elif method == 'CN':
            cc = nx.connected_component_subgraphs(g_copy)
            temp = 0.0
            for c in cc:
                n_c = nx.number_of_nodes(c)
                temp += n_c * (n_c-1) / 2
#             robust = 1 - temp / norm
                robust = temp / norm
        Robust.append(robust)
    return Robust

In [11]:
POS_Krebs = {0: array([-0.08519189,  0.49990594]),
 1: array([-0.08362047, 0.12333364]),
 2: array([-0.36443316, -0.51293337]),
 3: array([ 0.6929783 , -0.27211311]),
 4: array([-0.75119199, -0.52217359]),
 5: array([-0.48151941,  0.77948341]),
 6: array([-0.98609367, 0.10574352]),
 7: array([ 0.40822881, -0.81945388]),
 8: array([0.60618778, 0.55879015]),
 9: array([ 0.60877418, -0.59333843]),
 10: array([ 0.8790202 , -0.43466233]),
 11: array([-0.07012465, -0.44854881]),
 12: array([-0.65984227, -0.22090057]),
 13: array([-0.49419762,  0.01550339]),
 14: array([1.03216183, 0.02887253]),
 15: array([1.09737497, 0.43573233]),
 16: array([0.55469813, 0.14313733]),
 17: array([0.30147786, 0.41049205]),
 18: array([-0.29457097,  0.36139937]),
 19: array([ 0.30023929, -0.06238236]),
 20: array([ 0.24062728, -0.45484967]),
 21: array([-0.59082626,  0.30523301]),
 22: array([-0.33034474, -0.88517343]),
 23: array([-0.97685729, -0.28578861]),
 24: array([-0.66452836, -1.36418078]),
 25: array([0.04315099, -0.93549604]),
 26: array([ 1.19437548, -0.32133816]),
 27: array([-1.12819514, -0.97452116]),
 28: array([-0.63343009, -0.76413465]),
 29: array([0.1177091 , 1.20334897]),
 30: array([-0.10769893,  0.91323295]),
 31: array([-0.16203342,  1.5379476 ]),
 32: array([-0.55456901,  1.17106771]),
 33: array([-0.39372903,  1.49195439]),
 34: array([-0.76608088,  1.41357757]),
 35: array([-0.97598766,  1.24214294]),
 36: array([-1.1651016 ,  1.10244847]),
 37: array([-1.20053212,  0.89940136]),
 38: array([-1.29173705,  0.72895589]),
 39: array([-1.45781053, -0.54776995]),
 40: array([ 0.11592969, -1.43775734]),
 41: array([ 0.52265298, -1.44381712]),
 42: array([ 0.69446048, -1.3455922 ]),
 43: array([ 0.80123183, -1.07565775]),
 44: array([-0.13647759, -1.28122741]),
 45: array([0.73013119, 1.07047407]),
 46: array([ 1.34951065, -0.75411256]),
 47: array([-1.0346906 , -0.54604152]),
 48: array([-1.25398617, -0.228956  ]),
 49: array([1.59569059, 0.86795019]),
 50: array([0.48048749, 0.99577729]),
 51: array([1.41073232, 0.09031587]),
 52: array([1.43560726, 0.44920241]),
 53: array([1.06648158, 0.87847438]),
 54: array([-0.46048341, -1.6250863 ]),
 55: array([-1.00575395, -1.28422973]),
 56: array([-0.4512219 , -1.13463633]),
 57: array([-0.13314034, -1.70846718]),
 58: array([1.15682996, 1.5421328 ]),
 59: array([2.        , 0.66110703]),
 60: array([1.89996131, 1.0678281 ]),
 61: array([1.64559263, 1.448528  ])}

In [17]:
method = 'CN'# 'CN'

data_test = '../data/Krebs.txt'
g_test = nx.read_edgelist(data_test)
g = nx.Graph()
for edge in g_test.edges():
    g.add_edge(int(edge[0]), int(edge[1]))

nodes = g.nodes()
NodeNum = nx.number_of_nodes(g)
g_copy = g.copy()
g_DQN = g.copy()

### read sol file
if method == 'CN':
    sol = [1, 19, 49, 20, 5, 14, 11, 15, 29, 7, 34, 22, 40, 25, 58, 43, 28, 27, 6, 12, 2, 30, 47, 52, 23, 32, 45, 3, 46, 57]
elif method == 'ND':
    sol = [1, 11, 5, 7, 22, 49, 14, 12, 20, 15, 27, 29, 6, 24, 40, 19, 34, 2, 43, 28, 58, 57, 47, 35, 44, 26, 45, 46, 33, 30, 13, 23, 53, 52]

sol_all = sol + list(set(nodes)^set(sol))
sol = sol_all[:len(sol)+1]
# sol = sol_all
### draw single png
DQN_Robust = getRobustness(g_DQN, sol, method) 

######### plot robust curve graph
fig, ax = plt.subplots()
x_DQN_robust = [i / NodeNum for i in range(len(sol))]
y_DQN_robust = DQN_Robust[:len(sol)]

ax.plot(x_DQN_robust, y_DQN_robust, color='black', marker='o', linestyle='-', label='FINDER')
ax.set_ylabel('Residual %s Connectivity'%method,labelpad = Labelpad, fontweight='bold', fontsize=LabelSize)
ax.set_xlabel('Fraction of Key-players', fontweight='bold', fontsize=LabelSize)
ax.set_title('ANC Curve - %s'%method, fontweight='bold',fontsize=LabelSize)

ax.set_xlim([-0.05,0.6])
ax.set_ylim([-0.05,1.05])

xmajorLocator = MultipleLocator(0.1) #将x主刻度标签设置为0.05的倍数
xmajorFormatter = FormatStrFormatter('%.1f') #设置x轴标签文本的格式
ax.xaxis.set_major_locator(xmajorLocator)
ax.xaxis.set_major_formatter(xmajorFormatter)

ymajorLocator = MultipleLocator(0.2) #将y轴主刻度标签设置为0.5的倍数
ymajorFormatter = FormatStrFormatter('%.1f') #设置y轴标签文本的格式
ax.yaxis.set_major_locator(ymajorLocator)
ax.yaxis.set_major_formatter(ymajorFormatter)

ax.tick_params(axis='x', labelsize=TickSize)
ax.tick_params(axis='y', labelsize=TickSize)
# ax.legend(loc='upper right', bbox_to_anchor=(1, 0.88), fontsize=LabelSize)
plt.close()
fig.savefig('../result/%s/ANC_%s.png'%(method, method), bbox_inches='tight')
# fig.savefig('./result/%s/ANC_%s.pdf'%(method, method), bbox_inches='tight')


######### plot snapshot
NodeScale = 300
MaxNodeSize = 400
MinNodeSize = 50
Nodesize_all = dict(nx.degree(g_copy))
for node in Nodesize_all.keys():
    temp = np.log10(Nodesize_all[node])*NodeScale
    if temp < MinNodeSize:
        temp = MinNodeSize
    Nodesize_all[node] = temp
Nodesize_all_list = [Nodesize_all[item] for item in g_copy.nodes()]

for i in range(len(sol)-1):
#     print ('Iter:%d'%i)
    fig, ax = plt.subplots()
    
    G = g.copy()
    Pos = POS_Krebs.copy()
    Node_size = Nodesize_all.copy()
    Sol = sol[:i+1]
    for node in Sol:
        G.remove_node(node)
        del Pos[node]
        del Node_size[node]
    NodeSize_list = [Node_size[item] for item in G.nodes()]
    Nodesize_Sol_list = [Nodesize_all[item] for item in Sol]
        
    nx.draw_networkx(g, pos=POS_Krebs, with_labels=False, node_color='mediumturquoise', node_size=Nodesize_Sol_list,
                     nodelist=Sol, edgelist=list(g.edges()), width=1, edge_color='silver', style='dashed')
    nx.draw_networkx(G, pos=Pos, with_labels=False, node_color='black', node_size=NodeSize_list, 
                     nodelist=list(G.nodes()), edgelist=list(G.edges()), width=2)
    
    ax.set_title('%d Key-players, %.1f%% Residual %s Connectivity'%(int(NodeNum*x_DQN_robust[i+1]),100*y_DQN_robust[i+1], method), 
                 pad = Labelpad, fontweight='bold', fontsize=LabelSize)
    
    plt.xticks([])
    plt.yticks([])
    plt.close()
    fig.savefig('../result/%s/%d.png'%( method, i), bbox_inches='tight')

print ('Closed!')

Closed!
